# Introduction à Semantic Kernel

Ce notebook vous permettra de découvrir **Semantic Kernel**, un SDK permettant d'intégrer et d'orchestrer des modèles de langage dans vos applications Python.  
Nous verrons notamment :
- L'installation et la configuration du SDK.
- La configuration du service LLM (OpenAI ou Azure OpenAI) via un fichier `.env`.
- Le chargement et l'utilisation du Kernel.
- L'exécution de fonctions sémantiques définies depuis des fichiers prompt et directement en code.
- La gestion d'une conversation (chat) en utilisant des arguments de Kernel pour garder l'historique.

Ce notebook s'adresse à des étudiants de niveau licence ayant quelques bases en Python.


In [4]:
# Installation de Semantic Kernel (si nécessaire)
%pip install --upgrade semantic-kernel

# Vérification de la version installée
from semantic_kernel import __version__
print(f"Semantic Kernel version : {__version__}")


Note: you may need to restart the kernel to use updated packages.
Semantic Kernel version : 1.20.0



### 📌 **Importation des bibliothèques nécessaires**
Dans cette cellule, nous allons importer les modules principaux.


In [5]:
import os
import json
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments  # Correction de l'import


## **📁 2. Chargement des paramètres de configuration**
### 📌 **Lecture des paramètres depuis un fichier `.env` ou JSON**
Le fichier de configuration `.env` doit contenir les clés nécessaires pour accéder aux services OpenAI/Azure OpenAI.

💡 **Vérifiez que vous avez bien créé un fichier `.env`** dans le même dossier que ce notebook avec ces valeurs :

```plaintext
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_CHAT_MODEL_ID="gpt-4o-mini"
```

👉 Nous allons maintenant **charger ces paramètres en Python** :

In [6]:
# Chargement du fichier .env
load_dotenv()

# Récupération des clés API et du modèle
llm_service = os.getenv("GLOBAL_LLM_SERVICE", "OpenAI")
api_key = os.getenv("OPENAI_API_KEY")
model_id = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini")

# Vérification
print(f"Service sélectionné : {llm_service}")
print(f"Modèle utilisé : {model_id}")


Service sélectionné : OpenAI
Modèle utilisé : gpt-4o-mini


In [12]:
# Importer le Kernel depuis Semantic Kernel
from semantic_kernel import Kernel

# Créer une instance du Kernel
kernel = Kernel()
print("Kernel initialisé avec succès.")


Kernel initialisé avec succès.


## Configuration du service LLM

Nous devons connecter notre Kernel à un service de Chat Completion.  
Pour cet exemple, nous allons utiliser OpenAI. Si vous préférez Azure OpenAI, adaptez le code en conséquence (voir la documentation).

La configuration se fait via l'ajout d'un service au Kernel.  
Assurez-vous que votre fichier `.env` contient votre clé API.


In [13]:
# Importation du service OpenAI pour le Chat Completion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# Ajout du service "default" au Kernel
kernel.add_service(OpenAIChatCompletion(service_id="default"))
print("Service OpenAI ajouté au Kernel.")


Service OpenAI ajouté au Kernel.


## Utiliser un Plugin de Prompt

Semantic Kernel permet de charger des **prompt plugins** stockés sur disque.  
Dans cet exemple, nous chargerons le plugin "FunPlugin" qui contient, par exemple, une fonction pour générer une blague.

Les fichiers du plugin (le prompt et sa configuration) sont stockés dans le répertoire `prompt_template_samples/`.  
Nous allons charger ce plugin et invoquer la fonction "Joke" pour générer une blague sur un sujet donné.


In [18]:
# Chemin correct vers les plugins
plugins_directory = "./prompt_template_samples/"

# Vérifier si le dossier du plugin existe avant de charger
if os.path.exists(os.path.join(plugins_directory, "FunPlugin")):
    fun_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="FunPlugin")
    joke_function = fun_plugin["Joke"]
    print("Plugin chargé avec succès.")
else:
    print("⚠️ Le plugin FunPlugin est introuvable. Vérifiez le chemin et assurez-vous qu'il est bien présent.")


# Invoquer la fonction pour générer une blague sur un thème donné
# Pour ce faire, nous utilisons des KernelArguments (ici, seul l'input est nécessaire)
from semantic_kernel.functions import KernelArguments

# Exemple : générer une blague sur "time travel to dinosaur age" avec un style "super silly"
joke_response = await kernel.invoke(joke_function, KernelArguments(input="time travel to dinosaur age", style="super silly"))
print("Blague générée :", joke_response)


Plugin chargé avec succès.
Blague générée : Why did the time traveler bring a ladder to the dinosaur age?

Because they heard the T-Rex was a little "short-tempered" and wanted to reach new heights in their friendship!


## Définir une fonction sémantique en ligne

Outre l'utilisation de plugins stockés sur disque, il est possible de définir des fonctions sémantiques directement dans votre code Python.  
Cette approche est particulièrement utile pour :
- Générer dynamiquement des prompts en fonction du contexte
- Prototyper rapidement des idées sans créer de fichiers séparés

Dans cet exemple, nous allons créer une fonction qui résume un texte donné en quelques mots (TL;DR).


In [15]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

# Définition du prompt
tldr_prompt = """
{{$input}}

Donne-moi un résumé en 5 mots ou moins.
"""

# Configuration de l'exécution
execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="default",
    ai_model_id=model_id,
    max_tokens=2000,
    temperature=0.7,
)

# Configuration du prompt template
tldr_template_config = PromptTemplateConfig(
    template=tldr_prompt,
    name="tldr",
    template_format="semantic-kernel",
    input_variables=[InputVariable(name="input", description="Texte à résumer", is_required=True)],
    execution_settings=execution_settings,
)

# Ajout de la fonction au Kernel
tldr_function = kernel.add_function(function_name="tldrFunction", plugin_name="tldrPlugin", prompt_template_config=tldr_template_config)

# Exécution de la fonction
async def run_tldr():
    input_text = "Demo était une poétesse grecque ancienne connue pour un unique poème gravé sur le Colosse de Memnon."
    tldr_summary = await kernel.invoke(tldr_function, KernelArguments(input=input_text))
    print("Résumé (TL;DR) :", tldr_summary)

# Lancer la fonction
await run_tldr()


Résumé (TL;DR) : Poétesse grecque, unique poème.


## Chat interactif avec le Kernel

Semantic Kernel offre également la possibilité de créer des interactions de type chatbot.  
Nous allons configurer une fonction de chat qui utilise des **Kernel Arguments** pour conserver l'historique de la conversation.

L'objectif est de simuler une conversation où l'utilisateur envoie un message, le bot y répond, et l'historique est mis à jour à chaque échange.


In [17]:
from semantic_kernel.contents import ChatHistory

# Initialiser l'historique
chat_history = ChatHistory()
chat_history.add_system_message("Vous êtes un chatbot utile et vous fournissez des recommandations de livres.")

# Définition du prompt de chat
chat_prompt = """
{{$history}}
User: {{$user_input}}
ChatBot:
"""

# Configuration de l'exécution
chat_exec_settings = OpenAIChatPromptExecutionSettings(
    service_id="default",
    ai_model_id=model_id,
    max_tokens=2000,
    temperature=0.7,
)

# Configuration du chat template
chat_template_config = PromptTemplateConfig(
    template=chat_prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="Message de l'utilisateur", is_required=True),
        InputVariable(name="history", description="Historique de la conversation", is_required=True),
    ],
    execution_settings=chat_exec_settings,
)

# Ajout au Kernel
chat_function = kernel.add_function(function_name="chat", plugin_name="chatPlugin", prompt_template_config=chat_template_config)

# Fonction asynchrone pour le chat
async def chat(input_text: str):
    print(f"Utilisateur : {input_text}")
    response = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))
    print(f"ChatBot : {response}")
    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(response))

# Exécution des exemples
await chat("Salut, je cherche des suggestions de livres sur la philosophie antique.")
await chat("Peux-tu m'en recommander quelques-uns ?")


Utilisateur : Salut, je cherche des suggestions de livres sur la philosophie antique.
ChatBot : Salut ! Voici quelques suggestions de livres sur la philosophie antique qui pourraient t'intéresser :

1. **"La République" de Platon** - Un dialogue majeur où Platon explore la justice, la politique et la nature de l'homme.

2. **"Éthique à Nicomaque" d'Aristote** - Ce livre est une réflexion sur l'éthique, la vertu et le bonheur, et il reste une référence incontournable dans l'étude de la philosophie morale.

3. **"Les Pensées" de Marc Aurèle** - Un recueil de réflexions personnelles de l'empereur romain qui illustre la philosophie stoïcienne et offre des conseils sur la vie et la vertu.

4. **"La Métaphysique" d'Aristote** - Un texte fondamental qui traite des questions de l'existence, de la substance et de la réalité.

5. **"Les Discours" d'Épictète** - Ce stoïcien aborde des thèmes de la liberté, du contrôle personnel et de la manière de vivre une vie vertueuse.

6. **"Les Maximes" de L

# Conclusion

Dans ce notebook, nous avons :
- Installé et configuré **Semantic Kernel**.
- Chargé et configuré le service LLM (ici, OpenAI).
- Utilisé un plugin de prompt pour générer une blague.
- Défini une fonction sémantique inline pour obtenir un résumé (TL;DR) d’un texte.
- Créé une interaction de type chatbot en gérant l'historique via **Kernel Arguments**.

Ces exemples illustrent comment Semantic Kernel peut vous aider à intégrer des capacités avancées de traitement du langage naturel dans vos applications Python de manière modulaire et flexible.

Pour aller plus loin, vous pouvez explorer :
- La création de plugins plus complexes.
- L'intégration de fonctions supplémentaires et la gestion d'historique persistant.
- L'utilisation de Semantic Kernel avec différents services LLM (Azure OpenAI, HuggingFace, etc.).

Bonne exploration et n'hésitez pas à poser des questions !
